In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv(r'/content/drive/MyDrive/train_all.csv', encoding='utf_8_sig')
test = pd.read_csv(r'/content/drive/MyDrive/test_all.csv', encoding='utf_8_sig')


In [4]:
train

,label,text,diff
0,positive,XSS 취약점 수정,diff --git a/adm/board_form_update.php b/adm/b...
1,negative,"Auto merge of #4754 - daira:fix-enable-debug, ...",diff --git a/depends/hosts/darwin.mk b/depends...
2,positive,CAMEL-9309: Make it easier to turn on|off java...,diff --git a/components/camel-ahc/src/main/jav...
3,positive,Fixed bug #72627: Memory Leakage In exif_proce...,diff --git a/ext/exif/exif.c b/ext/exif/exif.c...
4,negative,Merge branch 'dev',diff --git a/.htaccess b/.htaccess\nindex 3293...
...,...,...,...
7073,positive,Restrict permissions on cinder configuration\n...,diff --git a/manifests/base.pp b/manifests/bas...
7074,positive,avformat/avidec: Limit formats in gab2 to srt ...,diff --git a/libavformat/avidec.c b/libavforma...
7075,positive,Fixed possible direct calls,diff --git a/phpmyfaq/admin/stat.ratings.php b...
7076,positive,"Reject qname's wirelength > 255, `chopOff()` h...",diff --git a/pdns/dnsparser.cc b/pdns/dnsparse...


In [5]:
len(train)

7078

In [6]:
train['label'].value_counts()

negative    4443
positive    2635
Name: label, dtype: int64

In [7]:
test['label'].value_counts()

negative    1904
positive    1130
Name: label, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,negative,[mjr] Allow to optinoally specify end date for...,diff --git a/doc/CHANGES b/doc/CHANGES\nindex ...,"[-0.09504235, -0.049959723, -0.010932095, -0.1..."
1,positive,Fixed memory leak when creating nested excepti...,diff --git a/ChangeLog b/ChangeLog\nindex 019b...,"[-0.003868403, 0.0019373484, -0.07806293, 0.25..."
2,negative,regenerate pyaff.c with a newer Cython\n\nCyth...,diff --git a/pyaff/pyaff.c b/pyaff/pyaff.c\nin...,"[-0.12354728, 0.039077323, 0.008539512, 0.0264..."
3,positive,Added some tests for CXF-4629\n\ngit-svn-id: h...,diff --git a/rt/ws/security/src/main/java/org/...,"[-0.17392707, 0.22975408, -0.044745743, 0.0834..."
4,positive,Selectively cache headers\n\nCo-authored-by: J...,diff --git a/imap/util.c b/imap/util.c\nindex ...,"[0.023296665, 0.034457836, -0.046119798, 0.077..."
...,...,...,...,...
3029,positive,Add accessDeniedHandler method to ExceptionHan...,diff --git a/config/src/main/java/org/springfr...,"[-0.024503198, 0.45559555, 0.058523644, -0.005..."
3030,negative,Merge pull request #1247 from jbonofre/KARAF-6...,diff --git a/shell/ssh/src/main/java/org/apach...,"[-0.06588035, 0.12500167, 0.07387892, -0.05972..."
3031,negative,Bump @popperjs/core from 2.5.4 to 2.6.0 in /fr...,diff --git a/frontend/package-lock.json b/fron...,"[-0.09501648, 0.03217733, -0.033742663, 0.0475..."
3032,negative,No one should care about ungifs.,diff --git a/README.md b/README.md\nindex fa21...,"[-0.07875048, 0.3826215, -0.0040257354, 0.0300..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression F1-score:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.8559657218193804
Logistic Regression Precision: 0.8549908148946601
Logistic Regression Recall: 0.8559657218193804
Logistic Regression Recall: 0.8551340180707208
